In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set()

In [2]:
from multiproduct import generate_dataset

In [3]:
nodes_sizes = [5, 10, 15, 20, 30, 50, 70, 100]
levels_sizes = [3, 3, 3, 4, 4, 5, 5, 5]
item_sizes = [1, 3, 5, 7]
datasets = []
for nodes, levels in zip(nodes_sizes, levels_sizes):
    for items in item_sizes:
        n_nodes, supplies, demands, costs, capacities = generate_dataset(
                nodes=nodes, levels=levels, total_supplies=[400]*items, 
                total_demands=[500]*items, transp_costs=(100,1000), 
                random_state=42)        
        datasets.append({
            'nodes': nodes,
            'items': items,
            'n_nodes': n_nodes, 
            'supplies': supplies, 
            'demands': demands, 
            'costs': costs, 
            'capacities': capacities
        })

In [26]:
from multiproduct import init_singleobjective_GA, run_singleobjective_GA

In [37]:
import pickle
with open('datasets/datasets.pkl', 'wb') as f:
    pickle.dump(datasets, f, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
datasets[0]

{'nodes': 5,
 'items': 1,
 'n_nodes': [1, 1, 3],
 'supplies': [[400, 100]],
 'demands': [[190, 110, 200, 0]],
 'costs': [array([[761]]), array([[408, 869, 443]])],
 'capacities': [[400], [558], [190, 110, 200]]}

In [28]:
%%time
dataset = datasets[0]
pop, hof, log = run_singleobjective_GA(dataset['n_nodes'], dataset['supplies'],
                        dataset['demands'], dataset['costs'],
                        dataset['capacities'], pop_size=100, n_generations=100,
                        n_solutions=20, crossover_p=0.5, mutation_p=0.2,
                        early_stopping_rounds=10)

gen	nevals	Avg   	Std    	Min   	Max   
0  	100   	534886	13642.3	499293	547554
1  	70    	505091	7456.28	499293	523720
2  	60    	499293	5.82077e-11	499293	499293
3  	64    	499293	5.82077e-11	499293	499293
4  	70    	499293	5.82077e-11	499293	499293
5  	73    	499293	5.82077e-11	499293	499293
6  	74    	499293	5.82077e-11	499293	499293
7  	74    	499293	5.82077e-11	499293	499293
8  	67    	499293	5.82077e-11	499293	499293
9  	77    	499293	5.82077e-11	499293	499293
10 	72    	499293	5.82077e-11	499293	499293
11 	74    	499293	5.82077e-11	499293	499293
12 	75    	499293	5.82077e-11	499293	499293
Early stopping...
CPU times: user 1.23 s, sys: 12.1 ms, total: 1.24 s
Wall time: 1.24 s


In [29]:
%%time
dataset = datasets[1]
pop, hof, log = run_singleobjective_GA(dataset['n_nodes'], dataset['supplies'],
                        dataset['demands'], dataset['costs'],
                        dataset['capacities'], pop_size=100, n_generations=100,
                        n_solutions=20, crossover_p=0.5, mutation_p=0.2,
                        early_stopping_rounds=10)

gen	nevals	Avg        	Std    	Min        	Max        
0  	100   	1.74512e+06	23546.2	1.70849e+06	1.81235e+06
1  	66    	1.71234e+06	3820.03	1.70849e+06	1.72833e+06
2  	69    	1.7078e+06 	1252.67	1.70479e+06	1.70853e+06
3  	72    	1.70497e+06	584.501	1.70479e+06	1.70849e+06
4  	78    	1.70479e+06	3.94394	1.70478e+06	1.70479e+06
5  	74    	1.70478e+06	1.88956	1.70478e+06	1.70479e+06
6  	72    	1.70478e+06	0      	1.70478e+06	1.70478e+06
7  	76    	1.70478e+06	0      	1.70478e+06	1.70478e+06
8  	77    	1.70478e+06	13.0098	1.70472e+06	1.70478e+06
9  	67    	1.70475e+06	26.053 	1.70472e+06	1.70478e+06
10 	77    	1.70472e+06	3.51328	1.70471e+06	1.70472e+06
11 	56    	1.70471e+06	0      	1.70471e+06	1.70471e+06
12 	72    	1.7047e+06 	40.8489	1.7046e+06 	1.70471e+06
13 	69    	1.7046e+06 	16.016 	1.7046e+06 	1.70471e+06
14 	67    	1.70458e+06	61.275 	1.70434e+06	1.7046e+06 
15 	66    	1.70445e+06	127.543	1.70434e+06	1.7046e+06 
16 	73    	1.70434e+06	0.24736	1.70434e+06	1.70434e+06
17 	75    

In [30]:
%%time
dataset = datasets[2]
pop, hof, log = run_singleobjective_GA(dataset['n_nodes'], dataset['supplies'],
                        dataset['demands'], dataset['costs'],
                        dataset['capacities'], pop_size=100, n_generations=300,
                        n_solutions=20, crossover_p=0.5, mutation_p=0.2,
                        early_stopping_rounds=20)

gen	nevals	Avg        	Std  	Min        	Max        
0  	100   	3.04308e+06	20313	2.99874e+06	3.10235e+06
1  	72    	3.00814e+06	9846.66	2.98912e+06	3.0287e+06 
2  	66    	2.9884e+06 	5248.49	2.98094e+06	2.99964e+06
3  	80    	2.97688e+06	5833.89	2.9625e+06 	2.98885e+06
4  	72    	2.96631e+06	3461.33	2.9625e+06 	2.97286e+06
5  	68    	2.96227e+06	463.737	2.96117e+06	2.9625e+06 
6  	72    	2.96105e+06	598.598	2.95987e+06	2.9625e+06 
7  	76    	2.95994e+06	281.188	2.95987e+06	2.96117e+06
8  	70    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
9  	62    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
10 	76    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
11 	65    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
12 	61    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
13 	70    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
14 	77    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
15 	68    	2.95987e+06	4.65661e-10	2.95987e+06	2.95987e+06
16 	74    	2.95987e+06	4.65661e-10	2.

In [31]:
%%time
dataset = datasets[3]
pop, hof, log = run_singleobjective_GA(dataset['n_nodes'], dataset['supplies'],
                        dataset['demands'], dataset['costs'],
                        dataset['capacities'], pop_size=100, n_generations=300,
                        n_solutions=20, crossover_p=0.5, mutation_p=0.2,
                        early_stopping_rounds=20)

gen	nevals	Avg        	Std    	Min        	Max        
0  	100   	3.02362e+06	32447.3	2.92683e+06	3.09721e+06
1  	69    	2.95635e+06	16191.1	2.92683e+06	2.9957e+06 
2  	67    	2.92579e+06	8533.48	2.90608e+06	2.95119e+06
3  	69    	2.91145e+06	8937.38	2.90608e+06	2.92683e+06
4  	71    	2.90483e+06	3075.9 	2.89396e+06	2.90608e+06
5  	72    	2.89604e+06	3928.99	2.89396e+06	2.9058e+06 
6  	73    	2.89396e+06	0      	2.89396e+06	2.89396e+06
7  	74    	2.89389e+06	210.191	2.89329e+06	2.89396e+06
8  	70    	2.89337e+06	205.699	2.89329e+06	2.89396e+06
9  	70    	2.89315e+06	298.903	2.89239e+06	2.89329e+06
10 	72    	2.89225e+06	411.891	2.89149e+06	2.89329e+06
11 	69    	2.89152e+06	165.466	2.89128e+06	2.89239e+06
12 	63    	2.89106e+06	666.887	2.88917e+06	2.89149e+06
13 	70    	2.88921e+06	755.174	2.88805e+06	2.89128e+06
14 	72    	2.88755e+06	957.877	2.88578e+06	2.88917e+06
15 	67    	2.88566e+06	349.3  	2.8841e+06 	2.88637e+06
16 	71    	2.88443e+06	839.673	2.88288e+06	2.88578e+06
17 	68    

In [32]:
%%time
dataset = datasets[4]
pop, hof, log = run_singleobjective_GA(dataset['n_nodes'], dataset['supplies'],
                        dataset['demands'], dataset['costs'],
                        dataset['capacities'], pop_size=100, n_generations=300,
                        n_solutions=20, crossover_p=0.5, mutation_p=0.2,
                        early_stopping_rounds=20)

gen	nevals	Avg   	Std    	Min   	Max   
0  	100   	481625	24463.7	413294	525345
1  	70    	432849	11561.5	413294	468456
2  	60    	413400	4147.23	406913	423496
3  	63    	405078	3835.64	397981	412955
4  	68    	397663	2291.56	390808	405020
5  	66    	392949	3156.48	390808	397981
6  	73    	390808	5.82077e-11	390808	390808
7  	66    	390783	51.7837    	390676	390808
8  	73    	390680	22.5175    	390676	390808
9  	66    	390676	5.82077e-11	390676	390676
10 	70    	390438	586.606    	388859	390676
11 	62    	388927	300.293    	388859	390615
12 	68    	388859	5.82077e-11	388859	388859
13 	65    	388460	1134.85    	385232	388859
14 	67    	385628	1125.75    	385232	388859
15 	65    	385220	37.776     	385100	385232
16 	69    	385092	91.5609    	384852	385232
17 	69    	384877	69.4658    	384852	385100
18 	78    	384849	7.32492    	384825	384852
19 	75    	384831	11.1847    	384825	384852
20 	70    	384822	8.1        	384798	384825
21 	63    	384780	52.9503    	384644	384825
22 	75    	38466

In [34]:
%%time
dataset = datasets[25]
pop, hof, log = run_singleobjective_GA(dataset['n_nodes'], dataset['supplies'],
                        dataset['demands'], dataset['costs'],
                        dataset['capacities'], pop_size=100, n_generations=300,
                        n_solutions=20, crossover_p=0.5, mutation_p=0.2,
                        early_stopping_rounds=20)

gen	nevals	Avg        	Std    	Min        	Max        
0  	100   	1.30576e+06	33849.7	1.23508e+06	1.41881e+06
1  	66    	1.24895e+06	10960.7	1.23508e+06	1.27892e+06
2  	71    	1.2325e+06 	4756.66	1.22044e+06	1.24356e+06
3  	66    	1.22085e+06	3430.93	1.21436e+06	1.23342e+06
4  	67    	1.21634e+06	1993.41	1.21382e+06	1.22044e+06
5  	69    	1.20986e+06	4113.59	1.20143e+06	1.21436e+06
6  	74    	1.20156e+06	2022.46	1.19897e+06	1.20603e+06
7  	70    	1.19825e+06	1359.53	1.19536e+06	1.19948e+06
8  	69    	1.19507e+06	829.554	1.19246e+06	1.19585e+06
9  	73    	1.19098e+06	2695.05	1.18646e+06	1.19421e+06
10 	71    	1.18566e+06	1223.27	1.18248e+06	1.18706e+06
11 	75    	1.18201e+06	1871.67	1.17812e+06	1.18596e+06
12 	61    	1.17768e+06	1506.33	1.1739e+06 	1.18071e+06
13 	69    	1.17416e+06	1053.67	1.172e+06  	1.17728e+06
14 	70    	1.1705e+06 	1957.64	1.16686e+06	1.17374e+06
15 	65    	1.16511e+06	2350.31	1.16171e+06	1.16867e+06
16 	66    	1.16111e+06	1393.29	1.15773e+06	1.16199e+06
17 	68    

KeyboardInterrupt: 